In [ ]:
def analyze_velocity_synchronization_with_laplacian(G, positions_history, dt, resolution=1, weight="weight", seed=42):
    """
    Analyze velocity synchronization in a network using the Laplacian matrix.

    This function computes velocity synchronization properties by leveraging the network Laplacian
    and spectral analysis. It examines overall velocity variance, algebraic connectivity (λ₂),
    and community-based velocity synchronization patterns.

    Parameters:
        G (networkx.Graph): The network graph representing node connectivity.
        positions_history (list of np.ndarray): List of node position matrices over time
                                               (shape: [iterations, num_nodes, dim]).
        dt (float): Time step used for numerical integration.
        resolution (float, optional): Resolution parameter for Louvain community detection. Default is 1.
        weight (str, optional): Edge weight attribute for community detection. Default is "weight".
        seed (int, optional): Random seed for Louvain partitioning. Default is 42.

    Returns:
        dict: Dictionary containing velocity synchronization analysis results:
            - "Algebraic connectivity (lambda_2)": The second smallest eigenvalue of the Laplacian matrix.
            - "Second eigenvector (v_2)": The corresponding eigenvector for λ₂, affecting synchronization.
            - "Overall velocity variance": Variance of velocity magnitudes across all nodes.
            - "Community velocity variances": Dictionary of variance values per detected community.
    """

    # Step 1: Compute velocity vectors at the final time step
    velocities = (positions_history[-1] - positions_history[-2]) / dt  # Velocity estimate using finite differences
    velocity_magnitudes = np.linalg.norm(velocities, axis=1)  # Compute magnitudes of velocity vectors

    # Step 2: Compute the Laplacian matrix of the network
    laplacian_matrix = nx.laplacian_matrix(G, weight=weight).toarray()  # Convert sparse matrix to dense array
    eigenvalues, eigenvectors = eigh(laplacian_matrix)  # Eigen decomposition of the Laplacian matrix

    # Step 3: Extract algebraic connectivity (λ₂) and its corresponding eigenvector
    lambda_2 = eigenvalues[1]  # The second smallest eigenvalue (Fiedler value)
    v_2 = eigenvectors[:, 1]  # The corresponding eigenvector (Fiedler vector)

    # Step 4: Compute overall velocity variance
    overall_velocity_variance = np.var(velocity_magnitudes)

    # Step 5: Detect communities using NetworkX's built-in Louvain algorithm
    louvain_partitions = list(nx.community.louvain_partitions(G, weight=weight, resolution=resolution, threshold=1e-07, seed=seed))

    # Extract the final partition (the best detected community structure)
    final_partition = louvain_partitions[-1]

    # Step 6: Compute velocity variance within each detected community
    community_velocity_variances = {}
    for community_id, nodes in enumerate(final_partition):  # Iterating over detected communities
        indices = [list(G.nodes).index(node) for node in nodes]  # Convert node labels to indices
        community_velocity_variances[community_id] = np.var(velocity_magnitudes[indices])  # Compute variance

    return {
        "Algebraic connectivity (lambda_2)": lambda_2,
        "Second eigenvector (v_2)": v_2.tolist(),
        "Overall velocity variance": overall_velocity_variance,
        "Community velocity variances": community_velocity_variances,
    }

In [ ]:
def analyze_phase_synchronization_with_laplacian(G, phase_history, resolution=1, weight="weight", seed=42):
    """
    Analyze phase synchronization in a network using the Laplacian matrix.

    This function evaluates phase synchronization properties by leveraging spectral graph
    analysis and community detection. It examines overall phase variance, algebraic connectivity (λ₂),
    and phase synchronization patterns within detected communities.

    Parameters:
        G (networkx.Graph): The network graph representing node connectivity.
        phase_history (numpy.ndarray): Array of node phase values over time
                                       (shape: [iterations, num_nodes]).
        resolution (float, optional): Resolution parameter for Louvain community detection. Default is 1.
        weight (str, optional): Edge weight attribute for community detection. Default is "weight".
        seed (int, optional): Random seed for Louvain partitioning. Default is 42.

    Returns:
        dict: Dictionary containing phase synchronization analysis results:
            - "Algebraic connectivity (lambda_2)": The second smallest eigenvalue of the Laplacian matrix.
            - "Second eigenvector (v_2)": The corresponding eigenvector for λ₂, affecting synchronization.
            - "Overall phase variance": Variance of phase values across all nodes.
            - "Community phase variances": Dictionary of variance values per detected community.
    """

    # Step 1: Extract the final time step's phase data
    final_phases = phase_history[-1]

    # Step 2: Normalize phases to be within the range [0, 2π]
    final_phases = np.mod(final_phases, 2 * np.pi)

    # Step 3: Compute the Laplacian matrix of the network
    laplacian_matrix = nx.laplacian_matrix(G, weight=weight).toarray()  # Convert sparse matrix to dense array
    eigenvalues, eigenvectors = eigh(laplacian_matrix)  # Eigen decomposition of the Laplacian matrix

    # Step 4: Extract algebraic connectivity (λ₂) and its corresponding eigenvector
    lambda_2 = eigenvalues[1]  # The second smallest eigenvalue (Fiedler value)
    v_2 = eigenvectors[:, 1]  # The corresponding eigenvector (Fiedler vector)

    # Step 5: Compute overall phase variance
    overall_phase_variance = np.var(final_phases)

    # Step 6: Detect communities using NetworkX's built-in Louvain algorithm
    louvain_partitions = list(nx.community.louvain_partitions(G, weight=weight, resolution=resolution, threshold=1e-07, seed=seed))

    # Extract the final partition (the best detected community structure)
    final_partition = louvain_partitions[-1]

    # Step 7: Compute phase variance within each detected community
    community_phase_variances = {}
    for community_id, nodes in enumerate(final_partition):  # Iterating over detected communities
        indices = [list(G.nodes).index(node) for node in nodes]  # Convert node labels to indices
        community_phase_variances[community_id] = np.var(final_phases[indices])  # Compute variance

    return {
        "Algebraic connectivity (lambda_2)": lambda_2,
        "Second eigenvector (v_2)": v_2.tolist(),
        "Overall phase variance": overall_phase_variance,
        "Community phase variances": community_phase_variances,
    }